In [1]:
import warnings
warnings.filterwarnings("ignore")
from cpr.Nivel import Nivel
from cpr.SqlDb import SqlDb
import cpr.information as info
import os
import datetime
import pandas as pd

# Crones

- Transformación de reflectividad a intensidad de radar de miel a local

    Para esto se usa <<sudo sshfs usuario@ip:remote_radar_path local_radar_path -o allow_other -o reconnect>>
    sshfs usa ssh para montar la carpeta donde está la lluvia de radar del servidos a local.


- Migración nivel a base de datos local
- Gráficas de nivel
- Reportes
    - Niveles de riesgo, 3 horas
    - Lluvia acumulada en las cuencas, 2 1/2 horas y 1/2 hora futura
    - reporte de lluvia y nivel, 2 1/2 horas y 1/2 hora futura
    - gifs, 2 1/2 horas y 1/2 hora futura
    - Resumen diario
    - Calidad

### Transformación de reflectividad a intensidad de radar de miel a local

In [2]:
start = pd.to_datetime('2018-09-27 11:20')
end   = pd.to_datetime('2018-09-27 20:40')

In [3]:
"python3 /home/mcano/dev/cpr/crones/Radar2RainConvStra.py 2018-10-03 2018-10-03 /home/mcano/dev/cprweb/src/media/radar/ /home/mcano/dev/cprweb/src/media/101_RadarClass/ -1 19:09 -2 19:39 -v"

'python3 /home/mcano/dev/cpr/crones/Radar2RainConvStra.py 2018-10-03 2018-10-03 /home/mcano/dev/cprweb/src/media/radar/ /home/mcano/dev/cprweb/src/media/101_RadarClass/ -1 19:09 -2 19:39 -v'

In [ ]:
fechaI=None
fechaF=None
rutaRadar=None
rutaNc = None
verbose = None
hora_1 = None
hora_2 = None
a_yuter=15
b_yuter=30
metodo = 'yuter'
extrapol=None
kwargs = {}


dateText = dt.datetime.now().strftime('%Y%m%d%H%M')
start =  kwargs.get('start',datetime.datetime.now() + datetime.timedelta(hours = 5) - datetime.timedelta(minutes = 15))
end =  kwargs.get('end',datetime.datetime.now() + datetime.timedelta(hours = 5) + datetime.timedelta(minutes = 15)
)
hora1 = start.strftime('%H:%M')  
hora2  = end.strftime('%H:%M')  
rutaRadar = self.data_path + 'radar'
rutaNc = self.data_path + '101_RadarClass'

#Si tiene que revisar en las carpetas de radar lo hace
if extrapol == False:
    datesDias = pd.date_range(fechaI,fechaF,freq='D')
    a = pd.Series(np.zeros(len(datesDias)),index=datesDias)
    a = a.resample('A').sum()
    Anos = [i.strftime('%Y') for i in a.index.to_pydatetime()]

    datesDias = [d.strftime('%Y%m%d') for d in datesDias.to_pydatetime()]

    ListDays = []
    ListRutas = []
    for d in datesDias:
            try:
                L = os.listdir(rutaRadar+d)
                L = [l for l in L if any(map(l.startswith,Anos)) and l.endswith('010_120.bin')]
                ListDays.extend(L)
                Ruta = [rutaRadar+d+'/'+l for l in L if any(map(l.startswith,Anos)) and l.endswith('010_120.bin')]
                ListRutas.extend(Ruta)
            except:
                pass
    ListDays.sort()
    ListRutas.sort()
    ListRutasFin = []
    # Obtiene la ruta final teniendo en cuenta la ruta del dia en las carpetas de radar
    for j,i in zip(ListRutas, ListDays):
        d = dt.datetime.strptime(i[:-12],'%Y%m%d%H%M')
        if d >= start and d<= end:
            ListRutasFin.append(j)
#Si es extrapolacion no tiene que revisar en las carpetas de radar
else:
    ListRutasFin = os.listdir(rutaRadar)
    ListRutasFin.sort()
    ListRutasFin = [rutaRadar + i for i in ListRutasFin]

#CONVERSION, CLASIFICACION Y GUARDADO
#Carga la cuenca del AMVA
Niter = len(ListRutasFin)
Conv = np.zeros(Niter)
Stra = np.zeros(Niter)
Total = np.zeros(Niter)
rad = radar.radar_process()

for c,l in enumerate(ListRutasFin):
    Total[c] = 0
    Conv[c] = 0
    Stra[c] = 0
    rad.read_bin(l)
    rad.Class2ConvStratiform(metodo=metodo, b_yuter = b_yuter, a_yuter = a_yuter)
    rad.DBZ2Rain()
    #Obtiene la ref media
    Conv[c] = rad.Z[rad.ConvStra == 2].sum() / 1e6
    Stra[c] = rad.Z[rad.ConvStra == 1].sum() / 1e6
    #guarda
    if extrapol == False:
        rad.save_rain_class(rutaNC+l[-24:-3]+'nc')
    else:
        rad.save_rain_class(rutaNC+l[-24:-4]+'_extrapol.nc')
    Total[c] = Conv[c] + Stra[c]
    if verbose:
        print('no verbose')

### estaciones en riesgo

In [35]:
self = Nivel(**info.LOCAL)
df = self.level_all(start,start+datetime.timedelta(minutes=10))
risks = self.risk_df(df)
in_risk = risks[risks.sum(axis=1)>0.0].sort_index(ascending=False).index

In [ ]:
def rain_report(self,date):
    '''
    Gets last topo-batimetry in db
    Parameters
    ----------
    x_sensor   :   x location of sensor or point to adjust topo-batimetry
    Returns
    ----------
    last topo-batimetry in db, DataFrame
    '''
    date = pd.to_datetime(date)
    start = date-datetime.timedelta(minutes=150)# 3 horas atras
    end = date+datetime.timedelta(minutes=30)
    #filepaths
    local_path = self.data_path
    remote_path = 'mcano@siata.gov.co:/var/www/mario/reportes_lluvia/'
    day_path = local_path + date.strftime('%Y%m%d')+'/'
    station_path = day_path+str(self.codigo)+'/'
    filepath = station_path+self.file_format(start,end)
    #make directories
    os.system('mkdir %s'%day_path)
    os.system('mkdir %s'%station_path)
    #rain
    vec = self.radar_rain(start,end,ext='.bin')
    current_vect = vec.drop(vec.loc[date:].index).sum().values/1000
    future_vect = vec.drop(vec.loc[:date].index).sum().values/1000
    # level
    mean_rain = self.radar_rain(start,end)*12.0
    series = self.level(start,date).resample('5min').mean()
    series[series>self.info.offset] = np.NaN
    series.index.name = ''
    level_cond = (series.dropna().size/series.size) < 0.05 # condición de nivel para graficar
    rain_cond = len(current_vect)==0.0
    if level_cond:
        print ('WARNING : Not enough level data')
    if rain_cond:
        print ('WARNING : Not rain in basin')
    if level_cond or rain_cond:
        pass
    else:
        #plots
        self.plot_rain_future(current_vect,future_vect,filepath = filepath+'_rain')
        self.plot_level_report(series,mean_rain,self.risk_levels)
        plt.gca().set_xlim(start,end)
        plt.savefig(filepath+'_level.png',bbox_inches='tight')
        self.rain_report_reportlab(filepath,date)
        os.system('ssh mcano@siata.gov.co "mkdir /var/www/mario/reportes_lluvia/%s"'%(date.strftime('%Y%m%d')))
        query = "rsync -r %s %s/%s/"%(filepath+'_report.pdf',remote_path+date.strftime('%Y%m%d'),self.codigo)
        os.system(query)
        
def siata_remote_data_to_transfer(start,end,*args,**kwargs):
    remote = cpr.Nivel(**cpr.info.REMOTE)
    codigos_str = '('+str(list(self.infost.index)).strip('[]')+')'
    parameters = tuple([codigos_str,self.fecha_hora_query(start,end)])
    df = remote.read_sql('SELECT * FROM datos WHERE cliente in %s and %s'%parameters)
    return df

def filter_data_to_transfer(self,start,end,local_path=None,remote_path=None,**kwargs):
    transfer = self.siata_remote_data_to_transfer(start,end,**kwargs)
    def convert(x):
        try:
            value = pd.to_datetime(x).strftime('%Y-%m-%d')
        except:
            value = np.NaN
        return value
    transfer['fecha'] = transfer['fecha'].apply(lambda x:convert(x))
    transfer = transfer.loc[transfer['fecha'].dropna().index]
    if local_path:
        transfer.to_csv(local_path)
        if remote_path:
            os.system('scp %s %s'%(local_path,remote_path))
    return transfer

def export_tables(self,tablas):
    for tabla in tablas:
        initia = datetime.datetime.now()
        filename = '%s.csv'%tabla
        local_path = '/media/nicolas/maso/Mario/data_migration/%s'%filename
        remote_path = "mcano@siata.gov.co:data_migration/%s"%filename
        self.read_sql('select * from %s'%tabla).to_csv(local_path)
        print(os.system('scp %s %s'%(local_path,remote_path)))
        print('filename:%s,took:%s'%(filename,datetime.datetime.now()-inicia))

### filter data to transfer

In [ ]:
daily = pd.date_range('2018-08-18',datetime.datetime.now())
sundays = daily[daily.strftime('%a') =='dom']
# arguments
for count in np.arange(0,len(sundays)-1):
    inicia = datetime.datetime.now()
    start = sundays[count]
    end   = sundays[count+1]
    date_format = '%Y-%m-%d'
    filename = 'weekly_level_%s_%s.csv'%(start.strftime(date_format),end.strftime(date_format))
    local_path = '/media/nicolas/maso/Mario/weekly_data/%s'%filename
    remote_path = "mcano@siata.gov.co:weekly_data/%s"%filename
    #filter_data_to_transfer(start,end,local_path,remote_path)
    print('filename:%s,took:%s'%(filename,datetime.datetime.now()-inicia))

In [ ]:
tables_remote = ['estaciones','aforo_nueva','levantamiento_aforo_nueva']

tables_local = ['estaciones_estaciones',
                'myusers_item',
                'myusers_section',
                'myusers_topo',
                'uploadfiles_uploadfilesmodel']